<a href="https://colab.research.google.com/github/dohyun1411/Quora-Insincere-Questions-Classification/blob/preprocessing1/very_simple_classifiers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Very simple classifiers
such as k-NN, Naive Bayes, SVM, Decision Tree.

In [ ]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


Please verify your input path.

In [ ]:
import os

# input_path = '/kaggle/input/quora-insincere-questions-classification'
input_path = '/content/drive/MyDrive/ColabNotebooks/QIQC/data'
train_path = os.path.join(input_path, 'train.csv')
test_path = os.path.join(input_path, 'test.csv')
embeddings_path = os.path.join(input_path, 'embeddings.zip')

In [ ]:
if os.path.exists(train_path): train_df = pd.read_csv(train_path)
else: train_df = pd.read_csv(train_path + '.zip')

if os.path.exists(test_path): test_df = pd.read_csv(train_path)
else: test_df = pd.read_csv(train_path + '.zip')

print('train shape:', train_df.shape)
print(train_df.head())
print()
print('test shape:', test_df.shape)
print(test_df.head())

train shape: (1306122, 3)
                    qid  ... target
0  00002165364db923c7e6  ...      0
1  000032939017120e6e44  ...      0
2  0000412ca6e4628ce2cf  ...      0
3  000042bf85aa498cd78e  ...      0
4  0000455dfa3e01eae3af  ...      0

[5 rows x 3 columns]

test shape: (1306122, 3)
                    qid  ... target
0  00002165364db923c7e6  ...      0
1  000032939017120e6e44  ...      0
2  0000412ca6e4628ce2cf  ...      0
3  000042bf85aa498cd78e  ...      0
4  0000455dfa3e01eae3af  ...      0

[5 rows x 3 columns]


In [ ]:
# fill up the missing values
trainX = train_df["question_text"].fillna("_na_").values
testX = test_df["question_text"].fillna("_na_").values

trainY = train_df["target"].values

Use NLTK word_toknize.

In [ ]:
import nltk
from nltk import word_tokenize
# nltk.download('punkt')

train_sents = []
for sent in tqdm(trainX): # 1min 21sec in Colab
    train_sents.append(word_tokenize(sent))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Features: (Please add more features)

1. len sent: length of the sentence
2. num words: number of words in the sentence
3. per capitals: percentage of capitalized words in the sentence
4. per no-alpha: percentage of non-alphabetic words in the sentence
5. per nouns: percentage of nouns in the sentence
6. polarity score: polarity score of the sentence
 

In [ ]:
feature_df = pd.DataFrame()

In [ ]:
len_sent = [len(sent) for sent in trainX]
feature_df['len sent'] = len_sent

num_words = [len(sent) for sent in train_sents]
feature_df['num words'] = num_words

per_capitals = [len([word for word in sent if word.isupper()]) / len(sent) for sent in train_sents]
feature_df['per capitals'] = per_capitals

per_no_alpha = [len([word for word in sent if not word.isalpha()]) / len(sent) for sent in train_sents]
feature_df['per no alpha'] = per_no_alpha

In [ ]:
# nltk.download('averaged_perceptron_tagger')

per_nouns = []
for sent in tqdm(train_sents): # 6min 57sec in Colab
    noun_sent = []
    for word, pos in nltk.pos_tag(sent):
        if pos.startswith('N'):
            noun_sent.append(word)
    per_nouns.append(len(noun_sent) / len(sent))
feature_df['per nouns'] = per_nouns

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

sid = SentimentIntensityAnalyzer()
polarity_score = []
for sent in tqdm(trainX):
    polarity_score.append(sid.polarity_scores(sent)['compound'])
feature_df['polarity score'] = polarity_score

In [ ]:
feature_df['target'] = trainY
feature_df

Visualize the features vs target.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

sns.scatterplot(x=feature_df['len sent'], y=feature_df['target'])
plt.title('len sent vs target')

In [ ]:
sns.scatterplot(x=feature_df['num words'], y=feature_df['target'])
plt.title('num words vs target')

In [ ]:
sns.scatterplot(x=feature_df['per capitals'], y=feature_df['target'])
plt.title('per capitals vs target')

In [ ]:
sns.scatterplot(x=feature_df['per no alpha'], y=feature_df['target'])
plt.title('per no alpha vs target')

In [ ]:
sns.scatterplot(x=feature_df['per nouns'], y=feature_df['target'])
plt.title('per nouns vs target')

In [ ]:
sns.scatterplot(x=feature_df['polarity score'], y=feature_df['target'])
plt.title('polarity score vs target')

Nothing interesting..

It seems like the results will be very poor. But we will do this as an experiment.

We will not hadle the outliers because of the bias of the data. The outliers might be an important feature.

In [ ]:
from sklearn.preprocessing import RobustScaler

def feature_scaling(df):

    target = df.pop('target') # We do not want to scale target.
    scaler = RobustScaler() # Use Robust Scaler for scaling.
    scaler.fit(df)
    df = pd.DataFrame(
        scaler.transform(df),
        columns=df.columns,
        index=list(df.index.values)
        )
    df['target'] = target # Merge the target value after scaling the feautres.

    return df

In [ ]:
feature_df = feature_scaling(feature_df)
feature_df

We will use scikit-learn.

In [ ]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(feature_df, test_size=0.2)
trainY = train_df['target']
trainX = train_df.drop(['target'], axis=1)
valY = val_df['target']
valX = val_df.drop(['target'], axis=1)

In [ ]:
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV

###Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

nb = GaussianNB()
nb.fit(trainX, trainY)
predY = nb.predict(valX)
f1_score(valY, predY)

f1 score = 0.2231, very poor..

Let's select some informative features.

In [ ]:
from sklearn.svm import SVC

def get_score(classifier_name, cur_features, tuning=False, verbose=0):
    if classifier_name == 'nb':
        nb = GaussianNB()
        nb.fit(trainX, trainY)
        predY = nb.predict(valX)
        return f1_score(valY, predY)

    if classifier_name == 'svm':
        if tuning:
            grid_params = {'C': [0.001, 0.1, 1, 10], 'kernel': ['poly', 'rbf', 'sigmoid', 'precomputed']}
            gs = GridSearchCV(SVC(), grid_params, verbose=(verbose > 1))
            gs.fit(trainX, trainY)

            svm_best_params = gs.best_params_
            print("svm best params:", svm_best_params)
        else:
            svm_best_params = {'C': 1, 'kernel': 'rbf'}
        
        svm = SVC(**svm_best_params)
        svm.fit(trainX, trainY)
        predY = svm.predict(valX)
        return f1_score(valY, predY)

In [ ]:
from copy import deepcopy

def forward_selection(n_features, classifier_name, tuning=False, verbose=1):

    remaining_features = trainX.columns.tolist()
    selected_features = []
    best_score = 0.

    prev_features = []
    for _ in tqdm(range(n_features), disable=(not verbose)):
        cur_best_score = 0.
        cur_best_feature = ''

        for feature in remaining_features:
            cur_features = prev_features + [feature]

            score = get_score(classifier_name, cur_features, tuning=tuning, verbose=verbose)
            if score > cur_best_score:
                cur_best_score = score
                cur_best_feature = feature

        prev_features.append(cur_best_feature)
        remaining_features.remove(cur_best_feature)

        if cur_best_score > best_score:
            selected_features = deepcopy(prev_features)
            best_score = cur_best_score
    
    return selected_features, best_score

In [ ]:
selected_features, best_score = forward_selection(6, 'nb')
print("selected features:", selected_features)
print("best f1 score:", best_score)

Nothing changed..

Let's try other classifiers.

###SVM

In [ ]:
selected_features, best_score = forward_selection(6, 'svm', verbose=2)
print("selected features:", selected_features)
print("best f1 score:", best_score)